In [1]:
# load data

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from time import time

from data_util import get_data_set, maybe_download_and_extract
from model_util import model

In [3]:
train_x, train_y, train_l = get_data_set(cifar=10)
test_x, test_y, test_l = get_data_set("test", cifar=10)

x, y, output, global_step, y_pred_cls = model()

_IMG_SIZE = 32
_NUM_CHANNELS = 3
_BATCH_SIZE = 256
_CLASS_SIZE = 10
_ITERATION = 5000
_SAVE_PATH = "/tmp/tensorboard/cifar-10/"


loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y))


starter_learning_rate = 1e-5
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           500, 0.9, staircase=True)

# Passing global_step to minimize() will increment it at each step.
optimizer = (
    tf.train.RMSPropOptimizer(learning_rate)
    .minimize(loss, global_step=global_step)
)
#optimizer = tf.train.RMSPropOptimizer(learning_rate=1e-5).minimize(loss, global_step=global_step)


correct_prediction = tf.equal(y_pred_cls, tf.argmax(y, dimension=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar("Loss/train", loss)
tf.summary.scalar("Accuracy/train", accuracy)



merged = tf.summary.merge_all()
saver = tf.train.Saver()
sess = tf.Session()
train_writer = tf.summary.FileWriter(_SAVE_PATH, sess.graph)



In [4]:
print(train_x.shape)
print(train_y.shape)
print(train_l[:5])
img_train_x = train_x.reshape(-1,32,32,3)
print(img_train_x.shape)

(50000, 3072)
(50000, 10)
['airplane', 'automobile', 'bird', 'cat', 'deer']
(50000, 32, 32, 3)


In [5]:
def train(num_iterations):
    '''
        Train CNN
    '''
    for i in range(num_iterations):
        randidx = np.random.randint(len(train_x), size=_BATCH_SIZE)
        batch_xs = train_x[randidx]
        batch_ys = train_y[randidx]

        start_time = time()
        i_global, _ = sess.run([global_step, optimizer], feed_dict={x: batch_xs, y: batch_ys})
        duration = time() - start_time

        if (i_global % 10 == 0) or (i == num_iterations - 1):
            _loss, batch_acc = sess.run([loss, accuracy], feed_dict={x: batch_xs, y: batch_ys})
            summary = tf.Summary(value=[
                tf.Summary.Value(tag="Loss/test", simple_value=_loss),
            ])
            train_writer.add_summary(summary, i_global)
            msg = "Global Step: {0:>6}, accuracy: {1:>6.1%}, loss = {2:.2f} ({3:.1f} examples/sec, {4:.2f} sec/batch)"
            print(msg.format(i_global, batch_acc, _loss, _BATCH_SIZE / duration, duration))

        if (i_global % 100 == 0) or (i == num_iterations - 1):
            
            
            data_merged, global_1 = sess.run([merged, global_step], feed_dict={x: batch_xs, y: batch_ys})
            acc = predict_test()

            summary = tf.Summary(value=[
                tf.Summary.Value(tag="Accuracy/test", simple_value=acc),
            ])
            train_writer.add_summary(data_merged, global_1)
            train_writer.add_summary(summary, global_1)

            saver.save(sess, save_path=_SAVE_PATH, global_step=global_step)
            print("Saved checkpoint.")

In [6]:
def predict_test(show_confusion_matrix=False):
    '''
        Make prediction for all images in test_x
    '''
    i = 0
    predicted_class = np.zeros(shape=len(test_x), dtype=np.int)
    while i < len(test_x):
        j = min(i + _BATCH_SIZE, len(test_x))
        batch_xs = test_x[i:j, :]
        batch_ys = test_y[i:j, :]
        predicted_class[i:j] = sess.run(y_pred_cls, feed_dict={x: batch_xs, y: batch_ys})
        i = j

    correct = (np.argmax(test_y, axis=1) == predicted_class)
    acc = correct.mean()
    correct_numbers = correct.sum()
    print("Accuracy on Test-Set: {0:.2f}% ({1} / {2})".format(acc*100, correct_numbers, len(test_x)))

    if show_confusion_matrix is True:
        cm = confusion_matrix(y_true=np.argmax(test_y, axis=1), y_pred=predicted_class)
        for i in range(_CLASS_SIZE):
            class_name = "({}) {}".format(i, test_l[i])
            print(cm[i, :], class_name)
        class_numbers = [" ({0})".format(i) for i in range(_CLASS_SIZE)]
        print("".join(class_numbers))

    return acc

In [ ]:
try:
    print("Trying to restore last checkpoint ...")
    last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=_SAVE_PATH)
    saver.restore(sess, save_path=last_chk_path)
    print("Restored checkpoint from:", last_chk_path)
except:
    print("Failed to restore checkpoint. Initializing variables instead.")
    sess.run(tf.global_variables_initializer())

Trying to restore last checkpoint ...
INFO:tensorflow:Restoring parameters from None
Failed to restore checkpoint. Initializing variables instead.


In [ ]:
if _ITERATION != 0:
    train(_ITERATION)

Global Step:     10, accuracy:   7.0%, loss = 2.30 (942.8 examples/sec, 0.27 sec/batch)
Global Step:     20, accuracy:  10.9%, loss = 2.30 (942.2 examples/sec, 0.27 sec/batch)
Global Step:     30, accuracy:  10.5%, loss = 2.30 (911.0 examples/sec, 0.28 sec/batch)
Global Step:     40, accuracy:   8.2%, loss = 2.30 (883.2 examples/sec, 0.29 sec/batch)
Global Step:     50, accuracy:  10.2%, loss = 2.30 (952.5 examples/sec, 0.27 sec/batch)
Global Step:     60, accuracy:   7.4%, loss = 2.31 (938.0 examples/sec, 0.27 sec/batch)
Global Step:     70, accuracy:   9.8%, loss = 2.30 (919.5 examples/sec, 0.28 sec/batch)
Global Step:     80, accuracy:  10.5%, loss = 2.30 (937.0 examples/sec, 0.27 sec/batch)
Global Step:     90, accuracy:  11.7%, loss = 2.30 (959.4 examples/sec, 0.27 sec/batch)
Global Step:    100, accuracy:   9.4%, loss = 2.30 (1024.4 examples/sec, 0.25 sec/batch)
Accuracy on Test-Set: 9.99% (999 / 10000)
Saved checkpoint.
Global Step:    110, accuracy:  10.2%, loss = 2.30 (914.1 e

Global Step:    890, accuracy:  27.7%, loss = 1.89 (883.9 examples/sec, 0.29 sec/batch)
Global Step:    900, accuracy:  27.0%, loss = 1.96 (1039.9 examples/sec, 0.25 sec/batch)
Accuracy on Test-Set: 26.52% (2652 / 10000)
Saved checkpoint.
Global Step:    910, accuracy:  31.6%, loss = 1.92 (968.9 examples/sec, 0.26 sec/batch)
Global Step:    920, accuracy:  25.4%, loss = 2.00 (942.1 examples/sec, 0.27 sec/batch)
Global Step:    930, accuracy:  36.3%, loss = 1.85 (1034.5 examples/sec, 0.25 sec/batch)
Global Step:    940, accuracy:  28.5%, loss = 1.98 (935.7 examples/sec, 0.27 sec/batch)
Global Step:    950, accuracy:  29.3%, loss = 1.95 (938.2 examples/sec, 0.27 sec/batch)
Global Step:    960, accuracy:  28.5%, loss = 1.96 (940.8 examples/sec, 0.27 sec/batch)
Global Step:    970, accuracy:  25.8%, loss = 1.95 (911.2 examples/sec, 0.28 sec/batch)
Global Step:    980, accuracy:  31.2%, loss = 1.89 (942.7 examples/sec, 0.27 sec/batch)
Global Step:    990, accuracy:  27.0%, loss = 1.98 (937.

Global Step:   1760, accuracy:  35.9%, loss = 1.74 (934.9 examples/sec, 0.27 sec/batch)
Global Step:   1770, accuracy:  32.4%, loss = 1.78 (910.8 examples/sec, 0.28 sec/batch)
Global Step:   1780, accuracy:  30.5%, loss = 1.77 (937.7 examples/sec, 0.27 sec/batch)
Global Step:   1790, accuracy:  32.8%, loss = 1.77 (1037.7 examples/sec, 0.25 sec/batch)
Global Step:   1800, accuracy:  34.0%, loss = 1.78 (946.6 examples/sec, 0.27 sec/batch)
Accuracy on Test-Set: 33.99% (3399 / 10000)
Saved checkpoint.
Global Step:   1810, accuracy:  33.6%, loss = 1.85 (914.3 examples/sec, 0.28 sec/batch)
Global Step:   1820, accuracy:  31.6%, loss = 1.82 (960.9 examples/sec, 0.27 sec/batch)
Global Step:   1830, accuracy:  31.2%, loss = 1.84 (1037.7 examples/sec, 0.25 sec/batch)
Global Step:   1840, accuracy:  28.1%, loss = 1.82 (937.8 examples/sec, 0.27 sec/batch)
Global Step:   1850, accuracy:  41.4%, loss = 1.69 (966.4 examples/sec, 0.26 sec/batch)
Global Step:   1860, accuracy:  34.0%, loss = 1.83 (1028

Global Step:   2630, accuracy:  36.7%, loss = 1.71 (920.4 examples/sec, 0.28 sec/batch)
Global Step:   2640, accuracy:  33.6%, loss = 1.83 (885.1 examples/sec, 0.29 sec/batch)
Global Step:   2650, accuracy:  30.9%, loss = 1.76 (915.1 examples/sec, 0.28 sec/batch)
Global Step:   2660, accuracy:  33.6%, loss = 1.70 (977.5 examples/sec, 0.26 sec/batch)
Global Step:   2670, accuracy:  35.5%, loss = 1.70 (1040.2 examples/sec, 0.25 sec/batch)
Global Step:   2680, accuracy:  42.6%, loss = 1.62 (1037.6 examples/sec, 0.25 sec/batch)
Global Step:   2690, accuracy:  40.2%, loss = 1.64 (1037.1 examples/sec, 0.25 sec/batch)
Global Step:   2700, accuracy:  39.1%, loss = 1.70 (934.9 examples/sec, 0.27 sec/batch)
Accuracy on Test-Set: 36.15% (3615 / 10000)
Saved checkpoint.
Global Step:   2710, accuracy:  36.7%, loss = 1.71 (900.3 examples/sec, 0.28 sec/batch)
Global Step:   2720, accuracy:  35.5%, loss = 1.73 (912.8 examples/sec, 0.28 sec/batch)
Global Step:   2730, accuracy:  38.7%, loss = 1.66 (937

In [ ]:
predict_test(True)

In [ ]:
g = tf.get_default_graph()
feed = g.get_tensor_by_name('data/images:0')
fetch = g.get_tensor_by_name('output/output:0')

# Feeding 3 images through the net just for testing
feed_vals = img_train_x[0:3]
res = sess.run(fetch, feed_dict={feed:feed_vals})
np.shape(feed_vals), res.shape

In [ ]:
# for embedding
N = 2000
p = 10
EMB = np.zeros((N, p), dtype='float32')
for i in range(N): #Of course you could do mini-batches
    EMB[i] = sess.run(fetch, feed_dict={feed: img_train_x[i:i+1,:]})
    if (i % 500 == 0 or i < 5):
        print(i)

In [ ]:
LOG_DIR = _SAVE_PATH
from tensorflow.contrib.tensorboard.plugins import projector
import os
# The embedding variable, which needs to be stored
# Note this must a Variable not a Tensor!
embedding_var = tf.Variable(EMB,  name='Embedding_of_output')
sess.run(embedding_var.initializer)
summary_writer = tf.summary.FileWriter(LOG_DIR)
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

# Comment out if you don't have metadata
embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')

# Comment out if you don't want sprites
embedding.sprite.image_path = os.path.join(LOG_DIR, 'sprite.png')
embedding.sprite.single_image_dim.extend([img_train_x.shape[1], img_train_x.shape[1]])

projector.visualize_embeddings(summary_writer, config)
saver = tf.train.Saver([embedding_var])
saver.save(sess, os.path.join(LOG_DIR, 'model2.ckpt'), 1)

In [ ]:
metadata_file = open(os.path.join(LOG_DIR, 'metadata.tsv'), 'w')
metadata_file.write('Name\tClass\n')
print(train_y[:5])
print(train_l)

print(train_l[np.argmax(train_y[0])])
print(train_l[np.argmax(train_y[1])])
print(train_l[np.argmax(train_y[2])])

for i in range(N):
    metadata_file.write('%06d\t%s\n' % (i, train_l[np.argmax(train_y[i])]))
metadata_file.close()

In [ ]:
# Taken from: https://github.com/tensorflow/tensorflow/issues/6322
def images_to_sprite(data):
    """Creates the sprite image along with any necessary padding

    Args:
      data: NxHxW[x3] tensor containing the images.

    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
    # Inverting the colors seems to look better for MNIST
    #data = 1 - data

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data

sprite = images_to_sprite(img_train_x[:N])

In [ ]:
import scipy
scipy.misc.imsave(os.path.join(LOG_DIR, 'sprite.png'), sprite)

In [ ]:
%ls -lhl /tmp/tensorboard/cifar-10

In [ ]:
#sess.close()